In [1]:
import numpy as np
import pandas as pd

In [ ]:

class NeuralNetwork:
    def __init__(self, layers_dim, learning_rate=0.01):
        """
        Initializes the neural network parameters.
        :param layers_dim: List containing the number of neurons in each layer.
        :param learning_rate: Learning rate for gradient descent.
        """
        self.param = {}  # Stores weights & biases
        self.cach = {}  # Stores intermediate values (A, Z) for backprop
        self.grads = {}  # Stores computed gradients
        self.layers = len(layers_dim)  # Total number of layers
        self.learning_rate = learning_rate  # Learning rate for updates
        
        # Initialize weights & biases
        for l in range(1, self.layers):
            self.param[f"w{l}"] = np.random.randn(layers_dim[l], layers_dim[l - 1]) * 0.01
            self.param[f"b{l}"] = np.zeros((layers_dim[l], 1))

    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))

    def relu(self, Z):
        return np.maximum(0, Z)

    def sigmoid_derivative(self, A):
        return A * (1 - A)  # Derivative of Sigmoid: A * (1 - A)

    def relu_derivative(self, Z):
        return (Z > 0).astype(float)  # Derivative of ReLU: 1 for Z > 0, else 0

    def forward_propagation(self, features):
        """
        Performs forward propagation and stores activations & weighted sums.
        :param features: Input features (n_features, m_samples)
        """
        self.cach["A0"] = features  # Input layer activations

        for l in range(1, self.layers):
            w, b = self.param[f"w{l}"], self.param[f"b{l}"]
            z = np.dot(w, self.cach[f"A{l - 1}"]) + b
            self.cach[f"Z{l}"] = z

            # Use ReLU for hidden layers, Sigmoid for output layer
            if l < self.layers - 1:
                self.cach[f"A{l}"] = self.relu(z)
            else:
                self.cach[f"A{l}"] = self.sigmoid(z)  # Output layer

        return self.cach[f"A{self.layers - 1}"]  # Return final predictions

    def compute_loss(self, Y, AL):
        """
        Computes the binary cross-entropy loss.
        :param Y: True labels (1, m_samples)
        :param AL: Predicted output (1, m_samples)
        """
        m = Y.shape[1]
        loss = -np.sum(Y * np.log(AL) + (1 - Y) * np.log(1 - AL)) / m
        return loss

    def backpropagation(self, Y):
        """
        Performs backpropagation to compute gradients for weights & biases.
        :param Y: True labels (1, m_samples)
        """
        m = Y.shape[1]  # Number of samples
        L = self.layers - 1  # Output layer index

        # Compute gradient of loss w.r.t. output activation
        dA = -(np.divide(Y, self.cach[f"A{L}"]) - np.divide(1 - Y, 1 - self.cach[f"A{L}"]))

        # Loop backward through layers
        for l in reversed(range(1, self.layers)):
            dZ = dA * (self.sigmoid_derivative(self.cach[f"A{l}"]) if l == L else self.relu_derivative(self.cach[f"Z{l}"]))
            self.grads[f"dW{l}"] = np.dot(dZ, self.cach[f"A{l-1}"].T) / m
            self.grads[f"db{l}"] = np.sum(dZ, axis=1, keepdims=True) / m
            dA = np.dot(self.param[f"w{l}"].T, dZ)  # Backprop to previous layer

    def update_parameters(self):
        """
        Updates weights & biases using gradient descent.
        """
        for l in range(1, self.layers):
            self.param[f"w{l}"] -= self.learning_rate * self.grads[f"dW{l}"]
            self.param[f"b{l}"] -= self.learning_rate * self.grads[f"db{l}"]

    def train(self, X, Y, epochs=1000):
        """
        Trains the neural network using forward propagation, loss computation, 
        backpropagation, and parameter updates.
        :param X: Input features (n_features, m_samples)
        :param Y: True labels (1, m_samples)
        :param epochs: Number of training iterations
        """
        for epoch in range(epochs):
            AL = self.forward_propagation(X)  # Forward pass
            loss = self.compute_loss(Y, AL)  # Compute loss
            self.backpropagation(Y)  # Backpropagation
            self.update_parameters()  # Update weights
            
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict(self, X):
        """
        Makes predictions using the trained model.
        :param X: Input features (n_features, m_samples)
        """
        AL = self.forward_propagation(X)
        return (AL > 0.5).astype(int)  # Convert probabilities to binary output
